<a href="https://colab.research.google.com/github/luricl/dogs_recognition/blob/dogFaceNet/dogfacenet/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DogFaceNet Training

Esse arquivo contém:
 - Split dos Dataset
 - Carregamento do Dataset
 - Definição do Modelo
 - Treinamento do Modelo

In [4]:
%%capture
!git clone https://github.com/UnB-CIS/caramelo.git -b dogFaceNet
%cd caramelo/dogfacenet

### Imports

In [5]:
import tensorflow as tf
import os
import numpy as np
import tensorflow.keras.backend as K

# funcoes para o treinamento da triplet loss
from online_training import *

### Config

In [7]:
PATH        = '../data'                    # Path to the directory of the saved dataset
PATH_SAVE   = '../output/history/'          # Path to the directory where the history will be stored
PATH_MODEL  = '../output/model/2025.08.19/' # Path to the directory where the model will be stored
SIZE        = (224,224,3)                   # Size of the input images
TEST_SPLIT  = 0.1                           # Test ratio
VAL_SPLIT = 0.1                             # Validation ratio
TRAIN_SPLIT = 1 - (TEST_SPLIT + VAL_SPLIT)  # Train ratio

LOAD_NET    = False                         # Load a network from a saved model? If True NET_NAME and START_EPOCH have to be precised
NET_NAME    = '2025.08.19.dogfacenet'       # Network saved name
START_EPOCH = 0                             # Start the training at a specified epoch
NBOF_EPOCHS = 1                             # Number of epoch to train the network
STEPS_PER_EPOCH = 300                       # Number of steps per epoch
VALIDATION_STEPS = 30                       # Number of steps per validation

### Split e Carregamento dos Dados

In [8]:
import zipfile
import os

# Verifica se o diretório de dados já existe
if not os.path.exists(PATH):
    # Extrai data.zip para o diretório de dados
    with zipfile.ZipFile(PATH+'.zip', 'r') as zip_ref:
        zip_ref.extractall(PATH)
    print('Dados extraídos com sucesso!')
else:
    print('Diretório de dados já existe. Pulando extração.')

Dados extraídos com sucesso!


In [9]:
assert os.path.isdir(PATH), '[Erro] O caminho fornecido para o dataset não existe.'

# carrega o dataset
filenames = np.empty(0)
labels = np.empty(0)
idx = 0
for root,dirs,files in os.walk(PATH):
    if len(files)>1:
        for i in range(len(files)):
            files[i] = root + '/' + files[i]
        filenames = np.append(filenames,files)
        labels = np.append(labels,np.ones(len(files))*idx)
        idx += 1
assert len(labels)!=0, '[Erro] Nenhum dado fornecido.'

print('Total de imagens: {:d}'.format(len(labels)))

nbof_classes = len(np.unique(labels))
print('Total de classes: {:d}'.format(nbof_classes))

Total de imagens: 8363
Total de classes: 1393


In [10]:
nbof_train = int(TRAIN_SPLIT*nbof_classes)
nbof_val = int(VAL_SPLIT*nbof_classes)

# cria matrizes booleanas que indicam quais classes(cachorros) pertencem a qual split
keep_train = np.less(labels, nbof_train)
keep_val = np.logical_and(np.greater_equal(labels, nbof_train), np.less(labels, nbof_train + nbof_val))
keep_test = np.greater_equal(labels, nbof_train + nbof_val)

# seleciona arquivos pertencentes a cada split
filenames_train = filenames[keep_train]
labels_train = labels[keep_train]
filenames_test = filenames[keep_test]
labels_test = labels[keep_test]
filenames_val = filenames[keep_val]
labels_val = filenames[keep_val]

## Definindo o Modelo

In [9]:
# definindo a funcao de perda
alpha = 0.3
def triplet(y_true,y_pred):

    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]

    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]

    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.less(ap+alpha,an)

In [10]:
if LOAD_NET:
    print('Loading model from {:s}{:s}.{:d}.h5 ...'.format(PATH_MODEL,NET_NAME,START_EPOCH))

    model = tf.keras.models.load_model(
        '{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,START_EPOCH),
        custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

    print('Done.')
else:
    from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Add, GlobalAveragePooling2D # type: ignore
    from tensorflow.keras.layers import Dropout, Flatten, Dense, Lambda, BatchNormalization # type: ignore

    print('Defining model {:s} ...'.format(NET_NAME))

    emb_size = 32

    inputs = Input(shape=SIZE)

    x = Conv2D(16, (7, 7), (2, 2), use_bias=False, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3,3))(x)

    for layer in [16,32,64,128,512]:

        x = Conv2D(layer, (3, 3), strides=(2,2), use_bias=False, activation='relu', padding='same')(x)
        r = BatchNormalization()(x)

        x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
        x = BatchNormalization()(x)
        r = Add()([r,x])

        x = Conv2D(layer, (3, 3), use_bias=False, activation='relu', padding='same')(r)
        x = BatchNormalization()(x)
        x = Add()([r,x])


    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(emb_size, use_bias=False)(x)
    outputs = Lambda(lambda x: K.l2_normalize(x,axis=-1), output_shape=(1, emb_size))(x)

    model = tf.keras.Model(inputs,outputs)

    model.compile(loss=triplet,
                optimizer='adam',
                metrics=[triplet_acc])

    print('Done.')

print(model.summary())

Defining model 2025.08.19.dogfacenet ...
Done.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      2,352 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 19, 19,    │      2,304 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 19, 19,    │      2,304 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 19, 19,    │      2,304 │ add[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 19, 19,    │          0 │ add[0][0],        │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 10, 10,    │      4,608 │ add_1[0][0]       │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        128 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 10, 10,    │      9,216 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        128 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 10, 10,    │          0 │ batch_normalizat

 Total params: 5,826,992 (22.23 MB)

 Trainable params: 5,822,448 (22.21 MB)

 Non-trainable params: 4,544 (17.75 KB)

None


## Treinamento

In [11]:
max_epoch = NBOF_EPOCHS + START_EPOCH

max_step = 300
max_step_val = 30
batch_size = 3*10

tot_loss_val = 0
mean_loss_val = 0

tot_acc_val = 0
mean_acc_val = 0

# Save
loss = []
val_loss = []
acc = []
val_acc = []

for epoch in range(START_EPOCH,max_epoch):

    step = 1

    tot_loss = 0
    mean_loss = 0

    tot_acc = 0
    mean_acc = 0

    # Training
    for images_batch,labels_batch in online_adaptive_hard_image_generator(
        filenames_train,
        labels_train,
        model,
        mean_acc,
        batch_size,
        nbof_subclasses=10
        ):


        h = model.train_on_batch(images_batch,labels_batch)
        tot_loss += h[0]
        mean_loss = tot_loss/step
        tot_acc += h[1]
        mean_acc = tot_acc/step

        hard_triplet_ratio = max(0,1.2/(1+np.exp(-10*mean_acc+5.3))-0.19)

        print(
            "Epoch: " + str(epoch) + "/" + str(max_epoch) +
            ", step: " + str(step) + "/" + str(max_step) +
            ", loss: " + str(mean_loss) +
            ", acc: " + str(mean_acc) +
            ", hard_ratio: " + str(hard_triplet_ratio)
        )
        print(
            "Val loss: " + str(mean_loss_val) +
            ", val acc: " + str(mean_acc_val)
        )

        if step == max_step:
            break
        step+=1

    loss += [mean_loss]
    acc += [mean_acc]

    # Validando
    step = 1

    tot_loss_val = 0
    mean_loss_val = 0

    tot_acc_val = 0
    mean_acc_val = 0

    for images_batch,labels_batch in image_generator(filenames_val,labels_val,batch_size,use_aug=False):
        h = model.test_on_batch(images_batch,labels_batch)

        tot_loss_val += h[0]
        mean_loss_val = tot_loss_val/step
        tot_acc_val += h[1]
        mean_acc_val = tot_acc_val/step

        if step == max_step_val:
            break
        step+=1

    val_loss += [mean_loss_val]
    val_acc += [mean_acc_val]

    # Save
    model.save('{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,epoch))
    history_ = np.array([loss,val_loss,acc,val_acc])
    np.save('{:s}{:s}.{:d}.npy'.format(PATH_SAVE,NET_NAME,epoch),history_)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 879ms/step
Epoch: 0/1, step: 1/300, loss: 1.3268418, acc: 0.6, hard_ratio: 0.61182535
Val loss: 0, val acc: 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step
Epoch: 0/1, step: 2/300, loss: 2.3745475, acc: 0.55, hard_ratio: 0.46980077
Val loss: 0, val acc: 0
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step
Epoch: 0/1, step: 3/300, loss: 2.6706553, acc: 0.5222222, hard_ratio: 0.3866784
Val loss: 0, val acc: 0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Epoch: 0/1, step: 4/300, loss: 2.888616, acc: 0.5104167, hard_ratio: 0.3514371
Val loss: 0, val acc: 0
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step
Epoch: 0/1, step: 5/300, loss: 3.0577521, acc: 0.50033337, hard_ratio: 0.3216471
Val loss: 0, val acc: 0
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Epoch: 0/1, step: 6/300, loss: 3.1373281, acc: 0.48638892, hard_ratio: 0.28120166
Val loss: 0, val acc: 0
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step
Epoch: 0/1, step: 7/300, loss: 3.1861453, acc: 0.47404766, hard_ratio: 0.24638924
Val loss: 0, val acc: 0
2/2 

KeyboardInterrupt: 